# Text 3: Latent Dirichlet allocation
**Internet Analytics - Lab 4**

---

**Group:** W

**Names:**

* Olivier Cloux
* Thibault Urien
* Saskia Reiss

---

#### Instructions

*This is a template for part 3 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [ ]:
# Given imports
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors
from scipy.sparse import find

# Import pickle to open processed courses
import pickle as pk
# Use Sparks sparse vectors
from pyspark.mllib.linalg import Vectors 

from lab04_helper import *

## Exercise 4.8: Topics extraction

In [ ]:
# Let's open the pickle to load it.
with open( "cidWithBag.txt", "rb" ) as f:
    processedCourses = pk.load( f, encoding="utf-8" )
    
# We also need the list of words
with open( "indexToWord", "rb" ) as f:
    uniqueWords = pk.load( f, encoding="utf-8" )

# Get the course indexes for the matrix
with open( "indexToCourse.txt", "rb" ) as f:
    courseIndices = pk.load( f, encoding="utf-8" )
    
# Get the TF-IDF matrix
TF_course_matrix = csr_matrix(np.load("TFIDF.npy"), shape=((len(uniqueWords), len(courseIndices))))

In [ ]:
# Transform the list of words into a spark vector usig TF-IDF
#### Use snippets Get value with index from RDD and Spar's sparse vectors
courses_values = sc.parallelize(processedCourses)
courses_index = courses_values.zipWithIndex()

# Create an RDD we will fill with our sparse vectors 
v_dim = len(uniqueWords)
vector_list = []

# Create the sparse vectors using the matrix.
for i in range (len(courseIndices)):
    values = find(TF_course_matrix.getcol(i))[-1]
    indices = (TF_course_matrix.getcol(i).nonzero())[0] # Could use first of find though
    v = Vectors.sparse(v_dim, indices, values)
    vector_list.append((i, v))
    
# In the RDD we need a tuple wit (index of document, corresponding SparseVector) then map(list)
#docs = courseIndices.zipWithIndex(vector_list)
#print(docs)

# Create the RDD (corpus) from the list
corpus = sc.parallelize(vector_list)
#print(corpus.take(2))
corpus = corpus.map(list)

# Cluster the documents into ten topics using LDA
ldaModel = LDA.train(corpus, k=10)

In [ ]:
# Gives a list of the 10 most linked words to our topics
topic_indices = ldaModel.describeTopics(maxTermsPerTopic=10)

# Extraire les indices (premier tuple) de la liste de mots
topic = 1
for i in topic_indices:
    print("Topic #", topic)
    [print(uniqueWords[i[0][j]],"-", i[1][j])  for j in range(10)]
    topic += 1

TODO : Topic labels AND compare to LSI.

## Exercise 4.9: Dirichlet hyperparameters

## Exercise 4.10: EPFL's taught subjects

## Exercise 4.11: Wikipedia structure